# 实现一个BP神经网络的7个步骤

1. 选择神经网络 结构
2. 随机 初始化权重
3. 实现 前向传播
4. 实现 损失函数 $J(\Theta)$
5. 实现反向传播算法并计算 偏微分 $\frac{\partial}{\partial
\Theta_{jk}^{(i)}}J(\Theta)$
6. 使用 梯度检查 并在检查后关闭
7. 使用梯度下降或其它优化算法和反向传播来 优化 $\Theta$ 的函数 $J(\Theta)$
iris中文指鸢尾植物，这里存储了其萼片和花瓣的长宽，一共4个属性，鸢尾植物又分三类。与之相对，iris里有两个属性iris.data，iris.target，data里是一个矩阵，每一列代表了萼片或花瓣的长宽，一共4列，每一列代表某个被测量的鸢尾植物，一共采样了150条记录，所以查看这个矩阵的形状iris.data.shape，返回

In [2]:

import numpy as np
from sklearn import datasets
iris=datasets.load_iris()
#random it
perm=np.random.permutation(iris.target.size)
iris.data=iris.data[perm]
iris.target=iris.target[perm]
print iris.data.shape
print iris.target.shape


(150, 4)
(150,)


In [3]:
#unique（）保留数组中不同的值，返回两个参数。
np.unique(iris.target)

array([0, 1, 2])

## 选择人工神经网络结构

### 神经网络架构图

## 随机初始化权重

### 网路权重随机初始化
初始化 $\Theta_{ij}^{(l)}$ 为 $[-\epsilon, \epsilon]$ 之间的随机值。
初始化函数可定义如下：

In [3]:
input_layer_size = 4
hidden_layer_size = 10
classes = 3
e=2.71828
def random_init(input_layer_size,hidden_layer_size,classes,init_epsilon=0.12):
    Theta_1=np.random.rand(hidden_layer_size,input_layer_size+1)*2*init_epsilon-init_epsilon
    Theta_2=np.random.rand(classes,hidden_layer_size+1)*2*init_epsilon-init_epsilon
    return (Theta_1,Theta_2)

In [4]:

random_init(input_layer_size, hidden_layer_size, classes, init_epsilon=0.12)

(array([[-0.08267547,  0.04164781,  0.09490538,  0.07785741,  0.05623299],
        [ 0.03771939, -0.01777384, -0.0807277 ,  0.02449977, -0.03531843],
        [ 0.03186725,  0.05770334,  0.05286388, -0.03057844,  0.10930007],
        [-0.04460161,  0.03932394,  0.05653844, -0.03683461,  0.10571919],
        [-0.04851769, -0.01026708, -0.05827354, -0.11523151,  0.11453877],
        [ 0.02945968,  0.07646253,  0.02454807, -0.05511934, -0.10696167],
        [ 0.05065804, -0.07838027, -0.09736674,  0.09586527,  0.0253025 ],
        [ 0.06978794,  0.09255081, -0.0292227 ,  0.06086073, -0.07872292],
        [-0.03981722,  0.089049  , -0.07662639,  0.05959605,  0.01333483],
        [-0.02863058,  0.03052268, -0.0261543 ,  0.02679231,  0.04782662]]),
 array([[-0.08143781, -0.02038464, -0.10593807, -0.0210057 ,  0.02358742,
          0.00494331,  0.01238254, -0.04765356, -0.07909502, -0.0609985 ,
         -0.03671531],
        [ 0.02082683,  0.02645296, -0.00937623,  0.02695417,  0.02471905,
   

# 实现前向传播

每一层都以此为公式计算下一层：
$$G_{{\Theta}^{(l)}(X)} =\frac{1}{1+e^{-X{({\Theta}^{(l)})}^T}}$$

注意添加偏差项。

In [5]:
#一个sigmoid函数
def g(theta,X):
    return 1/(1+np.e ** (-np.dot(X,np.transpose(theta))))

前向传播函数
h = g(Theta_1, np.hstack((np.ones((X.shape[0], 1)), X)))
这个函数运行完大概是下面的效果
[[ 0  1  2  3  4]

 [ 5  6  7  8  9]
 
 [10 11 12 13 14]]
 
print np.hstack((np.ones((a.shape[0],1)),a))

[[  1.   0.   1.   2.   3.   4.]

[  1.   5.   6.   7.   8.   9.]

[  1.  10.  11.  12.  13.  14.]]
应该是为了增加偏偏执项WX+B

In [6]:
def predict(Theta_1,Theta_2,X):
    h = g(Theta_1, np.hstack((np.ones((X.shape[0], 1)), X)))
    o = g(Theta_2, np.hstack((np.ones((h.shape[0], 1)), h)))
    return o

# 实现损失函数

## 损失的数学表示

$$
J(\Theta) = -\frac{1}{m}[\sum{i=1}^m
\sum{k=1}^Kyk^{(i)}\log(h\Theta(x^{(i)}))_k + (1 -
yk^{(i)})\log(1-h\Theta(x^{(i)}))k)] + \frac{\lambda}{2m}\sum{l=1}^{L-1}\sum
_{i=1}^{sl}\sum{j=1}^{s{l+1}}(\Theta{ji}^{(l)})^2
$$
$K$是输出层单元个数, $L$是层数，我们这里是三， $s_l$ 是层 $l$ 中单元数。


In [7]:
def costfunction(nn_param, input_layer_size, hidden_layer_size, classes, X, y, lamb_da):
    Theta_1 = nn_param[:hidden_layer_size * (input_layer_size + 1)].reshape((hidden_layer_size, (input_layer_size + 1)))#nn_param[:2]代表选取前两行
    Theta_2 = nn_param[hidden_layer_size * (input_layer_size + 1):].reshape((classes, (hidden_layer_size + 1)))#nn_params[3:]前三列
    m = X.shape[0]
    y_recoded = np.eye(classes)[y,:]#取第y行
       # calculate regularator
    regularator = (np.sum(Theta_1**2) + np.sum(Theta_2 ** 2)) * (lamb_da/(2*m))#l1正则化
    a3=predict(Theta_1,Theta_2,X)
    J = 1.0 / m * np.sum(-1 * y_recoded * np.log(a3)-(1-y_recoded) * np.log(1-a3)) + regularator
    # print J
    return J



**因为scipy中优化函数要求输入参数是向量而不是矩阵，因此我们的函数实现也必须能够随时展开和复原矩阵。**

# 实现反向传播

## 正向传播计算网络输出
- 反向计算每一层的误差$\sigma^{(l)}$
- 由误差累加计算得到成本函数的偏微分$\frac{\partial}{\partial\Theta_{ij}^{(l)}}J(\Theta)$
- 同样，在实现函数时因为要传递给scipy.optimize模块中的优化函数，必须能展开矩阵参数为向量并可随时复原。

## 反向传播网络
> - 对训练例$(x_k,y_k)$,假定神经网络的输出为$y_{k真} =(y_{1真} , y_{2真} , y_{3真},\cdots)$

> - 即$y_{j测}=f({\beta}_j-{\theta}_j)$ *式１*

> - $y_{j真}$ 代表真实值，也就是标准答案,$y_{j测}$代表这个神经网路输出的值，他两个的差即为误差

> - $E_k=\frac{1}{2}\sum_{i=1}^{n}{(y_{j真}-y_{j测})}^2$ *式2*

> - 对任意参数的更新为$w=w-\Delta w*\eta$其中$\eta$为学习率，由人设定

> - 周志华那本书的原文是bp算法是以目标的负梯度方向对参数进行调整的，用的是梯度下降法,周志华书的附录B.4有梯度下降法.

## 对参数的更新

> - 对于给定学习率 $\Delta=-\eta{\frac{\partial E_k}{\partial w_{hj}}}$  *式3*
> - 第j个神经元的输入${\beta}_j=\sum_{h=1}^{q}w_{hj}b_h$  *式4*
> - 注意到w_hj先影响到第j个输出层神经元的输入值$\beta_j$,在影响到输出值${y_{j测}^k}$,然后再影响到$E_k$ *式5*
> - 因此可以有$\frac{\partial E_k}{\partial w_{hj}} = \frac{\partial E_k}{\partial y_{j测}^k} \cdot \frac{ \partial y_{j测}^k}{\partial \beta_j}\cdot \frac{\partial \beta_j }{\partial w_{hj}}$  *式6*

> - $\frac{\partial \beta_j }{\partial w_{hj}}=b_h$ //由上面的$\beta$公式可知 *式7*

> - 式１和式2可得 $g_j=-\frac{\partial E_k}{\partial y_{j测}^k} \cdot \frac{\partial  y_{j测}^k}{\partial \beta_j}$

> - =$-(y_{j测}^k-y_{j真}^k)f'(\beta_j-\theta_j)$

> - =$y_{j测}^k(1-y_{j测}^k)(y_{j真}^K-y_{j测}^k)$    sigmoid导数s'(x)=s(x)(1-s(x))

> - 因此$\Delta w_{hj}=\eta g_j b_h$


In [8]:


def Gradient(nn_param, input_layer_size, hidden_layer_size, classes, X, y, lamb_da):
    Theta_1 = nn_param[:hidden_layer_size * (input_layer_size + 1)].reshape((hidden_layer_size, (input_layer_size + 1)))
    Theta_2 = nn_param[hidden_layer_size * (input_layer_size + 1):].reshape((classes, (hidden_layer_size + 1)))
    
    m = X.shape[0]
    Theta1_grad = np.zeros(Theta_1.shape);#先置0
    Theta2_grad = np.zeros(Theta_2.shape);
    for t in range(m):
    # For the input layer, where l=1:
    # add a bias unit and forward propagate
        a1 = np.hstack((np.ones((X[t:t+1,:].shape[0], 1)), X[t:t+1,:]))#加入了偏置
        # For the hidden layers, where l=2:
        a2 = g(Theta_1, a1)
        a2 = np.hstack((np.ones((a2.shape[0], 1)), a2))
        a3 = g(Theta_2, a2)

        yy = np.zeros((1, classes))
        yy[0][y[t]] = 1
        # For the delta values:
        delta_3 = a3 - yy;
        delta_2=delta_3.dot(Theta_2)*(a2*(1-a2))#Theta_2相当于bh,也就是权重,因为是第二层，所以是a2
        print 'delta_2_my',delta_2,'\n delta_3_my',delta_3
        delta_2 = delta_2[0][1:] # Taking of the bias row
        delta_2 = np.transpose(delta_2)
        delta_2 = delta_2.reshape(1, (np.size(delta_2)))
        Theta1_grad = 1.0 / m * Theta1_grad + lamb_da / m * np.hstack((np.zeros((Theta_1.shape[0], 1)), Theta_1[:, 1:]))
        Theta2_grad = 1.0 / m * Theta2_grad + lamb_da / m * np.hstack((np.zeros((Theta_2.shape[0], 1)), Theta_2[:, 1:]))
 #       if t==10:
 #           print 'delta_1',delta_1,'\n delta_2',delta_2,'\n delta_3',delta_3,'Theta1_grad\n',Theta1_grad,'\nTheta2_grad\n',Theta2_grad,
        Grad = np.concatenate((Theta1_grad.ravel(), Theta2_grad.ravel()), axis=0)
        return Grad
    

根据公式$g_j=y_{j测}^k(1-y_{j测}^k)(y_{j真}^K-y_{j测}^k)$

$w_{hj}=\eta g_j b_h$

Theta_2应该为$b_h$

因为是第二层，所以是$a_2$

$delta3=(y_{j真}^K-y_{j测}^k)$

# 梯度检测

> - “But back prop as an algorithm has a lot of details and,you know, can be a
> - little bit tricky to implement.”
> - Andrew NG如是说
> - 即使看上去成本函数在一直减小，可能神经网络还是有问题。
> - 梯度检测能让你确认，哦，我的实现还正常。

# 梯度检测原理

简单的微分近似。但相比反向传播神经网络算法计算量更大。
$$
\frac{\partial}{\partial\theta{i}}J(\theta) \approx
\frac{J(\theta{i}+\epsilon) - J(\theta_{i}+\epsilon) }{2\epsilon}
$$

In [9]:
# define compute_grad
def compute_grad(theta, input_layer_size, hidden_layer_size, classes, X, y, lamb_da, epsilon=1e-4):
    n = np.size(theta)
    gradaproxy = np.zeros(n)
    for i in range(n):
        theta_plus = np.copy(theta) # Important for in numpy assign is shalow copy
        theta_plus[i] = theta_plus[i] + epsilon
        theta_minus = np.copy(theta)
        theta_minus[i] = theta_minus[i] - epsilon
        gradaproxy[i] = (costfunction(theta_plus, input_layer_size, hidden_layer_size, classes, X, y, lamb_da) - costfunction(theta_minus, input_layer_size, hidden_layer_size, classes, X, y, lamb_da)) / (2.0 * epsilon)
    return gradaproxy

# 把一切放到一起

### 先定义训练函数如下：

In [10]:
from scipy import optimize

def train(input_layer_size, hidden_layer_size, classes, X, y, lamb_da):
    Theta_1, Theta_2 = random_init(input_layer_size, hidden_layer_size, classes, init_epsilon=0.12)
    nn_param = np.concatenate((Theta_1.ravel(), Theta_2.ravel()))
    final_nn = optimize.fmin_cg(costfunction, 
                                np.concatenate((Theta_1.ravel(), Theta_2.ravel()), axis=0), 
                                fprime=Gradient, 
                                args=(input_layer_size,
                                      hidden_layer_size, 
                                      classes, 
                                      X, 
                                      y,
                                      lamb_da))
    return final_nn

# 使用训练集进行训练

## 我选取鸢尾花数据集的100条作为训练集，剩下50条作为测试集

In [11]:
X = iris.data[:100]
y = iris.target[:100]
lamb_da = 1.0 # must be float
input_layer_size = 4
hidden_layer_size =10
classes = 3

final_nn = train(input_layer_size, hidden_layer_size, classes, X, y, lamb_da)

delta_2_my [[-0.         -0.00278727 -0.00261358  0.00857087 -0.00838076  0.00650289
   0.03673299  0.01053331 -0.01038492 -0.00980543  0.0032509 ]] 
 delta_3_my [[ 0.50820666 -0.44842117  0.5691816 ]]
delta_2_my [[-0.         -0.00274661 -0.00260795  0.00850276 -0.00833915  0.00646839
   0.03637284  0.01047266 -0.01027713 -0.00972589  0.00320678]] 
 delta_3_my [[ 0.50815011 -0.44854753  0.56853998]]
delta_2_my [[-0.         -0.00258625 -0.00258206  0.00822486 -0.00816435  0.00632299
   0.03492914  0.01021745 -0.00984622 -0.00940442  0.00303111]] 
 delta_3_my [[ 0.50793317 -0.44907481  0.56596835]]
delta_2_my [[-0.         -0.00198194 -0.00242172  0.00703112 -0.00732831  0.00562317
   0.02911107  0.00899987 -0.00812891 -0.00806424  0.00234664]] 
 delta_3_my [[ 0.50721328 -0.45154442  0.55560635]]
delta_2_my [[-0.         -0.00104967 -0.00187393  0.00459264 -0.00523795  0.00389938
   0.01844476  0.00608422 -0.00506048 -0.00539806  0.00123411]] 
 delta_3_my [[ 0.50651079 -0.45758579  0.5

   0.00903395  0.00302705 -0.00243939 -0.00278288  0.00047302]] 
 delta_3_my [[ 0.50652875 -0.46467556  0.51973068]]
delta_2_my [[-0.         -0.00041622 -0.00107101  0.00224821 -0.00277875  0.00198233
   0.00894239  0.00299623 -0.00241429 -0.00275608  0.00046687]] 
 delta_3_my [[ 0.50653181 -0.46475314  0.51956618]]
delta_2_my [[-0.         -0.0004059  -0.00105231  0.00220224 -0.00272613  0.0019429
   0.0087593   0.00293458 -0.00236411 -0.00270241  0.00045464]] 
 delta_3_my [[ 0.5065381  -0.4649088   0.51923716]]
delta_2_my [[-0.         -0.0003855  -0.00101452  0.00211025 -0.00262035  0.00186384
   0.00839317  0.00281119 -0.00226387 -0.00259476  0.00043049]] 
 delta_3_my [[ 0.50655135 -0.46522214  0.51857906]]
delta_2_my [[-0.         -0.00034558 -0.00093742  0.00192616 -0.00240659  0.00170496
   0.0076611   0.00256419 -0.00206382 -0.00237817  0.00038348]] 
 delta_3_my [[ 0.5065805  -0.46585686  0.51726262]]
delta_2_my [[-0.         -0.00034146 -0.00092921  0.00190682 -0.00238397  0.

delta_2_my [[ -0.00000000e+00  -5.81350484e-05  -2.06555196e-04   3.82290770e-04
   -5.02961031e-04   3.43264514e-04   1.52664711e-03   5.02918534e-04
   -4.06907697e-04  -4.90764626e-04   5.98761071e-05]] 
 delta_3_my [[ 0.50696448 -0.47161659  0.50619319]]
delta_2_my [[ -0.00000000e+00  -5.56018346e-05  -1.98144578e-04   3.66319334e-04
   -4.82205906e-04   3.28959992e-04   1.46298110e-03   4.81820087e-04
   -3.89894887e-04  -4.70471403e-04   5.72080388e-05]] 
 delta_3_my [[ 0.50696978 -0.47168058  0.50607789]]
delta_2_my [[ -0.00000000e+00  -5.05642434e-05  -1.81272508e-04   3.34386727e-04
   -4.40642047e-04   3.00349923e-04   1.33566314e-03   4.39658011e-04
   -3.55884358e-04  -4.29844533e-04   5.19167865e-05]] 
 delta_3_my [[ 0.50698046 -0.47180882  0.50584731]]
delta_2_my [[ -0.00000000e+00  -5.00375280e-05  -1.79497005e-04   3.31034607e-04
   -4.36273707e-04   2.97345805e-04   1.32229585e-03   4.35233718e-04
   -3.52314421e-04  -4.25575584e-04   5.13646805e-05]] 
 delta_3_my [[ 0

In [12]:
grad_aprox = compute_grad(final_nn, input_layer_size, hidden_layer_size, classes, X, y, lamb_da)
grad_bp = Gradient(final_nn, input_layer_size, hidden_layer_size, classes, X, y, lamb_da)
print (grad_aprox - grad_bp) < 1e-1

delta_2_my [[ -0.00000000e+00  -1.01764090e-05  -3.83660742e-05   6.94790063e-05
   -9.23729592e-05   6.25130171e-05   2.77923918e-04   9.10575845e-05
   -7.39148388e-05  -8.99924340e-05   1.02585352e-05]] 
 delta_3_my [[ 0.50707337 -0.47288792  0.5039301 ]]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
 False  True False  True False  True False  True False  True  True]


# 对测试集使用训练得来的参数

## 可以看到，测试集中50个样本有_个判定错误，其它_个分类正确。

In [13]:
def test(final_nn, input_layer_size, hidden_layer_size, classes, X, y, lamb_da):
    Theta_1 = final_nn[:hidden_layer_size * (input_layer_size + 1)].reshape((hidden_layer_size, input_layer_size + 1))
    Theta_2 = final_nn[hidden_layer_size * (input_layer_size + 1):].reshape((classes, hidden_layer_size + 1))
    nsuccess = np.sum(np.argmax(predict(Theta_1, Theta_2, X), axis=1) == y)
    return nsuccess

n = test(final_nn, input_layer_size, hidden_layer_size, classes, iris.data[100:], iris.target[100:], lamb_da)
print n
n = test(final_nn, input_layer_size, hidden_layer_size, classes, iris.data[:100], iris.target[:100], lamb_da)
print n

10
40
